# Mount Drive/smm4h

In [1]:
from google.colab import drive
GOOGLE_DRIVE_MOUNT = "/content/gdrive"
drive.mount(GOOGLE_DRIVE_MOUNT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/smm4h

/content/gdrive/My Drive/smm4h


# Github commands

In [0]:
! git remote set-url origin https://<username>:<password>@github.com/toskn/nru_hse_team_hlpl2020.git

In [4]:
! git pull origin master --allow-unrelated-histories

From https://github.com/toskn/nru_hse_team_hlpl2020
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# ! git config --global user.email "kuzannagood@gmail.com"
# ! git config --global user.name "kuzanna2016"

In [0]:
# ! git add *

In [0]:
# ! git commit -m "add original_data files for DeepPavlov"

[master 899df14] add original_data files for DeepPavlov
 4 files changed, 7616 insertions(+), 1 deletion(-)
 create mode 100644 original_data/dp_test.csv
 create mode 100644 original_data/dp_train.csv
 create mode 100644 original_data/dp_val.csv
 rewrite train_networks.ipynb (81%)


In [0]:
# ! git push --set-upstream origin master

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 540.04 KiB | 2.97 MiB/s, done.
Total 7 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/toskn/nru_hse_team_hlpl2020.git
   59ab27a..899df14  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


# Read and preparate data

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from tqdm import tqdm_notebook

DATA_FOLDER = '/content/gdrive/My Drive/smm4h/original_data/'
data_train = pd.read_csv(DATA_FOLDER + 'task2_ru_training.tsv', sep='\t')
data_val = pd.read_csv(DATA_FOLDER + 'task2_ru_validation.tsv', sep='\t')

Train = data_train[['tweet', 'class']]
Train.fillna('', inplace=True)

Val = data_val[['tweet', 'class']]
Val.fillna('', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [0]:
Train['class'].value_counts()

0    5557
1     533
Name: class, dtype: int64

In [0]:
# with open("tfidf.pkl", 'wb') as handle:
#     pickle.dump(tf_idf, handle)

X_train = Train['tweet']
y_train = Train['class']

X_val = Val[['tweet']]
y_val = Val['class']

---------------------

In [7]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.5MB/s 


In [8]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub

# Models

## BERT 1.0

In [0]:
DATA_COLUMN = 'tweet'
LABEL_COLUMN = 'class'
label_list = [0, 1]

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = Train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = Val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# We'll set sequences to be at most 140 tokens long.
MAX_SEQ_LENGTH = 140
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 6090


INFO:tensorflow:Writing example 0 of 6090


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())
  
  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
      
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 4.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpz0mdjmni', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f391310c390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpz0mdjmni', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f391310c390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
from datetime import datetime

print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.0/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 761 into /tmp/tmpz0mdjmni/model.ckpt.


INFO:tensorflow:Saving checkpoints for 761 into /tmp/tmpz0mdjmni/model.ckpt.


INFO:tensorflow:Loss for final step: 0.031431545.


INFO:tensorflow:Loss for final step: 0.031431545.


Training took time  0:08:36.003605


In [0]:
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.0/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 761: auc = 0.6364967, eval_accuracy = 0.90735877, f1_score = 0.36771291, false_negatives = 92.0, false_positives = 49.0, global_step = 761, loss = 0.2738636, precision = 0.45555556, recall = 0.30827066, true_negatives = 1340.0, true_positives = 41.0


INFO:tensorflow:Saving dict for global step 761: auc = 0.6364967, eval_accuracy = 0.90735877, f1_score = 0.36771291, false_negatives = 92.0, false_positives = 49.0, global_step = 761, loss = 0.2738636, precision = 0.45555556, recall = 0.30827066, true_negatives = 1340.0, true_positives = 41.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 761: /tmp/tmpz0mdjmni/model.ckpt-761


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 761: /tmp/tmpz0mdjmni/model.ckpt-761


{'auc': 0.6364967,
 'eval_accuracy': 0.90735877,
 'f1_score': 0.36771291,
 'false_negatives': 92.0,
 'false_positives': 49.0,
 'global_step': 761,
 'loss': 0.2738636,
 'precision': 0.45555556,
 'recall': 0.30827066,
 'true_negatives': 1340.0,
 'true_positives': 41.0}

## DeepPavlov RuBERT

In [4]:
!pip install deeppavlov

     |████████████████████████████████| 757kB 4.6MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 2.1MB 36.5MB/s 
     |████████████████████████████████| 51kB 9.8MB/s 
     |████████████████████████████████| 2.9MB 51.5MB/s 
     |████████████████████████████████| 1.5MB 58.7MB/s 
     |████████████████████████████████| 8.0MB 20.8MB/s 
     |████████████████████████████████| 51kB 10.3MB/s 
     |████████████████████████████████| 7.3MB 57.9MB/s 
     |████████████████████████████████| 20.1MB 156kB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 6.7MB 53.0MB/s 
     |████████████████████████████████| 61kB 11.2MB/s 
     |████████████████████████████████| 7.1MB 15.3MB/s 
     |████████████████████████████████| 2.3MB 43.1MB/s 
     

In [6]:
!python -m deeppavlov install insults_kaggle_bert
!python -m deeppavlov download insults_kaggle_bert

     |████████████████████████████████| 757kB 2.7MB/s 
     |████████████████████████████████| 7.3MB 8.6MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 20.1MB 55.0MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 6.7MB 38.9MB/s 
     |████████████████████████████████| 8.0MB 35.2MB/s 
     |████████████████████████████████| 2.9MB 40.6MB/s 
     |████████████████████████████████| 1.5MB 30.9MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 2.1MB 37.8MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 7.1MB 40.1MB/s 
     |████████████████████████████████| 256kB 42.6MB/s 
     |███

2020-03-24 13:59:31.978 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'insults_kaggle_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/classifiers/insults_kaggle_bert.json'
     |████████████████████████████████| 110.5MB 96kB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.2 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-af352kf0
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-af352kf0
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'


In [0]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader

In [15]:
from sklearn.model_selection import train_test_split

new_train, new_val = train_test_split(Train, test_size=0.1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  


In [0]:
new_train.to_csv('/content/gdrive/My Drive/smm4h/original_data/dp_train.csv')
new_val.to_csv('/content/gdrive/My Drive/smm4h/original_data/dp_val.csv')
Val.to_csv('/content/gdrive/My Drive/smm4h/original_data/dp_test.csv')

In [0]:
reader = BasicClassificationDatasetReader()
data = reader.read(data_path="/content/gdrive/My Drive/smm4h/original_data", 
                   train="dp_train.csv", valid="dp_val.csv", test="dp_test.csv",
                   x="tweet", y="class")

In [0]:
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

In [0]:
iterator = BasicClassificationDatasetIterator(data, seed=42, shuffle=True)

In [0]:
from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor

In [0]:
!python -m deeppavlov download paraphraser_rubert

2020-03-22 11:59:39.604 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'paraphraser_rubert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/classifiers/paraphraser_rubert.json'
2020-03-22 11:59:39.807 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/datasets/paraphraser.zip?config=paraphraser_rubert to /root/.deeppavlov/downloads/paraphraser.zip
100% 1.16M/1.16M [00:00<00:00, 4.73MB/s]
2020-03-22 11:59:40.52 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/downloads/paraphraser.zip archive into /root/.deeppavlov/downloads/paraphraser_data
2020-03-22 11:59:40.277 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz?config=paraphraser_rubert to /root/.deeppavlov/paraphraser_rubert_v0.tar.gz
100% 659M/659M [02:42<00:00, 4.05MB/s]
2020-03-22 12:02:23.138 INF

In [0]:
bert_preprocessor = BertPreprocessor(vocab_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
                                     do_lower_case=False,
                                     max_seq_length=64)

In [0]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

In [21]:
vocab = SimpleVocabulary(save_path="./binary_classes.dict")
vocab.fit(iterator.get_instances(data_type="train")[1])

2020-03-24 18:31:14.136 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead


In [0]:
from deeppavlov.models.preprocessors.one_hotter import OneHotter

In [0]:
one_hotter = OneHotter(depth=vocab.len, 
                       single_vector=True  # means we want to have one vector per sample
                      )

In [0]:
from deeppavlov.models.classifiers.proba2labels import Proba2Labels

prob2labels = Proba2Labels(max_proba=True)

In [0]:
from deeppavlov.models.bert.bert_classifier import BertClassifierModel

In [0]:
bert_classifier = BertClassifierModel(
    n_classes=vocab.len,
    return_probas=True,
    one_hot_labels=True,
    bert_config_file="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
    pretrained_bert="~/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
    save_path="sst_bert_model/model",
    load_path="sst_bert_model/model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0
)

2020-03-22 13:54:02.52 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt]
INFO:deeppavlov.models.bert.bert_classifier:[initializing model with Bert from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt


In [0]:
from deeppavlov.metrics.accuracy import sets_accuracy
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [0]:
# Method `get_instances` returns all the samples of particular data field
x_valid, y_valid = iterator.get_instances(data_type="valid")
# You need to save model only when validation score is higher than previous one.
# This variable will contain the highest accuracy score
best_score = 0.45
patience = 4
impatience = 0

# let's train for 4 epochs
for ep in range(4):
  
    nbatches = 0
    for x, y in iterator.gen_batches(batch_size=64, 
                                     data_type="train", shuffle=True):
        x_feat = bert_preprocessor(x)
        y_onehot = one_hotter(vocab(y))
        bert_classifier.train_on_batch(x_feat, y_onehot)
        nbatches += 1
        
        if nbatches % 15 == 0:
            # валидируемся каждые 15 батчей
            y_valid_pred = bert_classifier(bert_preprocessor(x_valid))
            score_acc = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
            score_f1 = f1_score(y_valid, vocab(prob2labels(y_valid_pred)), pos_label="1")
            print("Batches done: {}. Valid Accuracy: {}. Valid F1: {}".format(nbatches, score_acc, score_f1))
            
    y_valid_pred = bert_classifier(bert_preprocessor(x_valid))
    score_acc = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
    score_f1 = f1_score(y_valid, vocab(prob2labels(y_valid_pred)), pos_label="1")
    y_test = bert_classifier(bert_preprocessor(Val['tweet']))
    score_f1_test = f1_score(Val['class'], prob2labels(y_test))
    score_acc_test = accuracy_score(Val['class'], prob2labels(y_test))
    print("Epochs done: {}. Test Accuracy: {}. Test F1: {}".format(ep + 1, score_acc_test, score_f1_test))
    if score_f1_test > best_score:
        bert_classifier.save()
        print("New best score. Saving model.")
        best_score = score_f1_test
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break

Batches done: 15. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 30. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 45. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 60. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 75. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Epochs done: 1. Test Accuracy: 0.9126149802890933. Test F1: 0.014814814814814812
Batches done: 15. Valid Accuracy: 0.8899835796387521. Valid F1: 0.4462809917355372
Batches done: 30. Valid Accuracy: 0.9064039408866995. Valid F1: 0.32941176470588235
Batches done: 45. Valid Accuracy: 0.9047619047619048. Valid F1: 0.30952380952380953
Batches done: 60. Valid Accuracy: 0.9146141215106732. Valid F1: 0.16129032258064516
Batches done: 75. Valid Accuracy: 0.909688013136289. Valid F1: 0.03508771929824561
Epochs done: 2. Test Accuracy: 0.9204993429697766. Test F1: 0.1879194630872483
Batches done: 15. Valid Accuracy: 0.9064039408866995. Valid F1: 0.5043478260869566

2020-03-22 13:56:59.772 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/sst_bert_model/model]
INFO:deeppavlov.core.models.tf_model:[saving model to /content/sst_bert_model/model]


Epochs done: 3. Test Accuracy: 0.871222076215506. Test F1: 0.46448087431693985
New best score. Saving model.
Batches done: 15. Valid Accuracy: 0.9064039408866995. Valid F1: 0.5439999999999999
Batches done: 30. Valid Accuracy: 0.8899835796387521. Valid F1: 0.5179856115107915
Batches done: 45. Valid Accuracy: 0.9113300492610837. Valid F1: 0.5423728813559322
Batches done: 60. Valid Accuracy: 0.9064039408866995. Valid F1: 0.4770642201834862
Batches done: 75. Valid Accuracy: 0.9064039408866995. Valid F1: 0.5043478260869566
Epochs done: 4. Test Accuracy: 0.9139290407358739. Test F1: 0.3350253807106599


## DeepPavlov Conversational RuBERT

In [0]:
!wget "http://files.deeppavlov.ai/deeppavlov_data/bert/ru_conversational_cased_L-12_H-768_A-12.tar.gz"

--2020-03-22 14:38:55--  http://files.deeppavlov.ai/deeppavlov_data/bert/ru_conversational_cased_L-12_H-768_A-12.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 660061308 (629M) [application/octet-stream]
Saving to: ‘ru_conversational_cased_L-12_H-768_A-12.tar.gz’

ru_conversational_c 100%[===================>] 629.48M  2.88MB/s    in 2m 26s  

2020-03-22 14:41:21 (4.31 MB/s) - ‘ru_conversational_cased_L-12_H-768_A-12.tar.gz’ saved [660061308/660061308]



In [0]:
!tar -xf ru_conversational_cased_L-12_H-768_A-12.tar.gz -C ~/.deeppavlov/downloads/bert_models/

In [0]:
bert_preprocessor_conv = BertPreprocessor(vocab_file="~/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/vocab.txt",
                                     do_lower_case=False,
                                     max_seq_length=64)

In [0]:
bert_classifier_conv = BertClassifierModel(
    n_classes=vocab.len,
    return_probas=True,
    one_hot_labels=True,
    bert_config_file="~/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/bert_config.json",
    pretrained_bert="~/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt",
    save_path="sst_bert_model_conv/model",
    load_path="sst_bert_model_conv/model",
    keep_prob=0.5,
    learning_rate=1e-05,
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0
)

2020-03-22 14:44:40.10 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt]
INFO:deeppavlov.models.bert.bert_classifier:[initializing model with Bert from /root/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/ru_conversational_cased_L-12_H-768_A-12/bert_model.ckpt


In [0]:
# Method `get_instances` returns all the samples of particular data field
x_valid, y_valid = iterator.get_instances(data_type="valid")
# You need to save model only when validation score is higher than previous one.
# This variable will contain the highest accuracy score
best_score = 0.45
patience = 4
impatience = 0

# let's train for 4 epochs
for ep in range(4):
  
    nbatches = 0
    for x, y in iterator.gen_batches(batch_size=64, 
                                     data_type="train", shuffle=True):
        x_feat = bert_preprocessor_conv(x)
        y_onehot = one_hotter(vocab(y))
        bert_classifier_conv.train_on_batch(x_feat, y_onehot)
        nbatches += 1
        
        if nbatches % 15 == 0:
            # валидируемся каждые 15 батчей
            y_valid_pred = bert_classifier_conv(bert_preprocessor_conv(x_valid))
            score_acc = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
            score_f1 = f1_score(y_valid, vocab(prob2labels(y_valid_pred)), pos_label="1")
            print("Batches done: {}. Valid Accuracy: {}. Valid F1: {}".format(nbatches, score_acc, score_f1))
            
    y_valid_pred = bert_classifier_conv(bert_preprocessor_conv(x_valid))
    score_acc = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
    score_f1 = f1_score(y_valid, vocab(prob2labels(y_valid_pred)), pos_label="1")
    y_test = bert_classifier_conv(bert_preprocessor_conv(Val['tweet']))
    score_f1_test = f1_score(Val['class'], prob2labels(y_test))
    score_acc_test = accuracy_score(Val['class'], prob2labels(y_test))
    print("Epochs done: {}. Test Accuracy: {}. Test F1: {}".format(ep + 1, score_acc_test, score_f1_test))
    if score_f1_test > best_score:
        bert_classifier_conv.save()
        print("New best score. Saving model.")
        best_score = score_f1_test 
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break

Batches done: 15. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 30. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 45. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 60. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Batches done: 75. Valid Accuracy: 0.9080459770114943. Valid F1: 0.0
Epochs done: 1. Test Accuracy: 0.9126149802890933. Test F1: 0.0
Batches done: 15. Valid Accuracy: 0.9146141215106732. Valid F1: 0.5094339622641509
Batches done: 30. Valid Accuracy: 0.8916256157635468. Valid F1: 0.5
Batches done: 45. Valid Accuracy: 0.909688013136289. Valid F1: 0.4444444444444445
Batches done: 60. Valid Accuracy: 0.8932676518883416. Valid F1: 0.5454545454545454
Batches done: 75. Valid Accuracy: 0.9113300492610837. Valid F1: 0.49056603773584906


2020-03-22 14:46:50.205 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/sst_bert_model_conv/model]
INFO:deeppavlov.core.models.tf_model:[saving model to /content/sst_bert_model_conv/model]


Epochs done: 2. Test Accuracy: 0.8791064388961892. Test F1: 0.4831460674157303
New best score. Saving model.
Batches done: 15. Valid Accuracy: 0.922824302134647. Valid F1: 0.5154639175257731
Batches done: 30. Valid Accuracy: 0.9129720853858785. Valid F1: 0.5309734513274336
Batches done: 45. Valid Accuracy: 0.9129720853858785. Valid F1: 0.49523809523809526
Batches done: 60. Valid Accuracy: 0.9178981937602627. Valid F1: 0.5
Batches done: 75. Valid Accuracy: 0.909688013136289. Valid F1: 0.4954128440366973
Epochs done: 3. Test Accuracy: 0.8876478318002629. Test F1: 0.5043478260869566
Batches done: 15. Valid Accuracy: 0.9211822660098522. Valid F1: 0.52
Batches done: 30. Valid Accuracy: 0.9244663382594417. Valid F1: 0.5660377358490566
Batches done: 45. Valid Accuracy: 0.9244663382594417. Valid F1: 0.4772727272727273
Batches done: 60. Valid Accuracy: 0.9261083743842364. Valid F1: 0.49438202247191004
Batches done: 75. Valid Accuracy: 0.9195402298850575. Valid F1: 0.5148514851485149
Epochs done

## 

## Pre-trained ELMO DeepPavlov

In [6]:
from deeppavlov import configs
from deeppavlov.core.commands.utils import parse_config
config_dict = parse_config(configs.classifiers.rusentiment_elmo_twitter_cnn)
print(config_dict['dataset_reader']['data_path'])

~/.deeppavlov/downloads/rusentiment/


In [7]:
!python -m deeppavlov install rusentiment_elmo_twitter_cnn
!python -m deeppavlov download rusentiment_elmo_twitter_cnn

2020-03-24 18:13:15.234 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'rusentiment_elmo_twitter_cnn' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/classifiers/rusentiment_elmo_twitter_cnn.json'
     |████████████████████████████████| 110.5MB 95kB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.2 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
2020-03-24 18:14:11.677 INFO in 'deeppavlov.core.common.file'['file'] at line 30: Interpreting 'rusentiment_elmo_twitter_cnn' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/classifiers/rusentiment_elmo_twitter_cnn.json'
2020-03-24 18:14:12.800 INFO in 'deeppavlov.core.data.utils'['utils'] at 

In [9]:
!ls ~/.deeppavlov/downloads/rusentiment/

ls: cannot access '/root/.deeppavlov/downloads/rusentiment/': No such file or directory


In [0]:
from deeppavlov.models.preprocessors.dirty_comments_preprocessor import DirtyCommentsPreprocessor

In [0]:
preprocessor = DirtyCommentsPreprocessor(remove_punctuation=False)

In [0]:
from deeppavlov.models.tokenizers.nltk_tokenizer import NLTKTokenizer

In [0]:
tokenizer = NLTKTokenizer()

In [0]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

In [33]:
embedder = ELMoEmbedder(spec='http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-twitter_2013-01_2018-04_600k_steps.tar.gz',
                        elmo_output_names=['elmo'],
                        mini_batch_size=32,
                        pad_zero=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
from deeppavlov.models.classifiers.keras_classification_model import KerasClassificationModel

In [53]:
config_dict

{'chainer': {'in': ['x'],
  'in_y': ['y'],
  'out': ['y_pred_labels'],
  'pipe': [{'class_name': 'simple_vocab',
    'fit_on': ['y'],
    'id': 'classes_vocab',
    'in': 'y',
    'load_path': '~/.deeppavlov/models/classifiers/rusentiment_v10/classes.dict',
    'out': 'y_ids',
    'save_path': '~/.deeppavlov/models/classifiers/rusentiment_v10/classes.dict'},
   {'class_name': 'dirty_comments_preprocessor',
    'in': ['x'],
    'out': ['x_prep'],
    'remove_punctuation': False},
   {'class_name': 'nltk_tokenizer',
    'id': 'my_tokenizer',
    'in': 'x_prep',
    'out': 'x_tok',
    'tokenizer': 'wordpunct_tokenize'},
   {'class_name': 'elmo_embedder',
    'elmo_output_names': ['elmo'],
    'id': 'my_embedder',
    'in': ['x_tok'],
    'mini_batch_size': 32,
    'out': ['x_emb'],
    'pad_zero': True,
    'spec': 'http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-twitter_2013-01_2018-04_600k_steps.tar.gz'},
   {'class_name': 'one_hotter',
    'depth': '#classes_vocab.len',
    'in': '

In [71]:
classifier = KerasClassificationModel(coef_reg_cnn=0.001,
                                      coef_reg_den=0.01,
                                      dense_size=100,
                                      dropout_rate=0.5,
                                      embedding_size=embedder.dim,
                                      filters_cnn=256,
                                      kernel_sizes_cnn=[3,5,7],
                                      last_layer_activation='softmax',
                                      learning_rate=0.01,
                                      learning_rate_decay=0.1,
                                      load_path='~/.deeppavlov/models/classifiers/rusentiment_v10/model',
                                      loss='categorical_crossentropy',
                                      main=True,
                                      model_name='cnn_model',
                                      n_classes=vocab.len,
                                      optimizer='Adam')

2020-03-24 19:31:09.214 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 245: [initializing `KerasClassificationModel` from saved]
INFO:deeppavlov.models.classifiers.keras_classification_model:[initializing `KerasClassificationModel` from saved]
2020-03-24 19:31:09.566 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 255: [loading weights from model.h5]
INFO:deeppavlov.models.classifiers.keras_classification_model:[loading weights from model.h5]
2020-03-24 19:31:09.820 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 129: Model was successfully initialized!
Model summary:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 

In [0]:
def preprocessing(x):
    x_prep = preprocessor(x)
    x_tok = tokenizer(x_prep)
    x_emb = embedder(x_tok)
    return x_emb

In [63]:
# Method `get_instances` returns all the samples of particular data field
x_valid, y_valid = iterator.get_instances(data_type="valid")
# You need to save model only when validation score is higher than previous one.
# This variable will contain the highest accuracy score
best_score = 0.45
patience = 5
impatience = 0

# let's train for 4 epochs
for ep in range(4):
  
    nbatches = 0
    for x, y in iterator.gen_batches(batch_size=32, 
                                     data_type="train", shuffle=True):
        x_prep = preprocessing(x)
        y_onehot = one_hotter(vocab(y))
        classifier.train_on_batch(x_prep, y_onehot)
        nbatches += 1
        
        if nbatches % 15 == 0:
            # валидируемся каждые 15 батчей
            y_valid_pred = classifier(preprocessing(x_valid))
            score_acc = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
            score_f1 = f1_score(y_valid, vocab(prob2labels(y_valid_pred)), pos_label="1")
            print("Batches done: {}. Valid Accuracy: {}. Valid F1: {}".format(nbatches, score_acc, score_f1))
            
    y_valid_pred = classifier(preprocessing(x_valid))
    score_acc = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
    score_f1 = f1_score(y_valid, vocab(prob2labels(y_valid_pred)), pos_label="1")
    y_test = classifier(preprocessing(Val['tweet']))
    score_f1_test = f1_score(Val['class'], prob2labels(y_test))
    score_acc_test = accuracy_score(Val['class'], prob2labels(y_test))
    print("Epochs done: {}. Test Accuracy: {}. Test F1: {}".format(ep + 1, score_acc_test, score_f1_test))
    if score_f1_test > best_score:
        classifier.save()
        print("New best score. Saving model.")
        best_score = score_f1_test
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break

InvalidArgumentError: ignored

## LSTM